In [1]:
import pandas as pd
import numpy as np
import os
from sklearn import tree
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
import pickle 
pd.set_option('display.max_columns', None)

In [2]:
# load in our data
big_df = pd.read_csv("../data/big_df.csv")
big_df

,lev1,blue_age,red_age,blue_heightIN,red_heightIN,blue_name,red_name,blue_reach,red_reach,blue_stance,red_stance,blue_cumsum_tot_stks_lnd,red_cumsum_tot_stks_lnd,blue_cumsum_tot_stks_att,red_cumsum_tot_stks_att,blue_cumsum_sig_stks_lnd,red_cumsum_sig_stks_lnd,blue_cumsum_sig_stks_att,red_cumsum_sig_stks_att,blue_cumsum_TDs_lnd,red_cumsum_TDs_lnd,blue_cumsum_TDs_att,red_cumsum_TDs_att,blue_cumsum_sub_att,red_cumsum_sub_att,blue_cumsum_ctrl_sec,red_cumsum_ctrl_sec,blue_cumsum_KDs,red_cumsum_KDs,blue_cumsum_revs,red_cumsum_revs,blue_cumsum_sig_body_lnd,red_cumsum_sig_body_lnd,blue_cumsum_sig_body_att,red_cumsum_sig_body_att,blue_cumsum_sig_head_lnd,red_cumsum_sig_head_lnd,blue_cumsum_sig_head_att,red_cumsum_sig_head_att,blue_cumsum_sig_leg_lnd,red_cumsum_sig_leg_lnd,blue_cumsum_sig_leg_att,red_cumsum_sig_leg_att,blue_cumsum_sig_dist_lnd,red_cumsum_sig_dist_lnd,blue_cumsum_sig_dist_att,red_cumsum_sig_dist_att,blue_cumsum_sig_clch_lnd,red_cumsum_sig_clch_lnd,blue_cumsum_sig_clch_att,red_cumsum_sig_clch_att,blue_cumsum_sig_gnd_lnd,red_cumsum_sig_gnd_lnd,blue_cumsum_sig_gnd_att,red_cumsum_sig_gnd_att,blue_cumsum_tot_stks_abs,red_cumsum_tot_stks_abs,blue_cumsum_tot_stks_seen,red_cumsum_tot_stks_seen,blue_cumsum_sig_stks_abs,red_cumsum_sig_stks_abs,blue_cumsum_sig_stks_seen,red_cumsum_sig_stks_seen,blue_cumsum_TDs_abs,red_cumsum_TDs_abs,blue_cumsum_TDs_seen,red_cumsum_TDs_seen,blue_cumsum_sub_esc,red_cumsum_sub_esc,blue_cumsum_ctrld_sec,red_cumsum_ctrld_sec,blue_cumsum_KedD,red_cumsum_KedD,blue_cumsum_reversed,red_cumsum_reversed,blue_cumsum_sig_body_abs,red_cumsum_sig_body_abs,blue_cumsum_sig_body_seen,red_cumsum_sig_body_seen,blue_cumsum_sig_head_abs,red_cumsum_sig_head_abs,blue_cumsum_sig_head_seen,red_cumsum_sig_head_seen,blue_cumsum_sig_leg_abs,red_cumsum_sig_leg_abs,blue_cumsum_sig_leg_seen,red_cumsum_sig_leg_seen,blue_cumsum_sig_dist_abs,red_cumsum_sig_dist_abs,blue_cumsum_sig_dist_seen,red_cumsum_sig_dist_seen,blue_cumsum_sig_clch_abs,red_cumsum_sig_clch_abs,blue_cumsum_sig_clch_seen,red_cumsum_sig_clch_seen,blue_cumsum_sig_gnd_abs,red_cumsum_sig_gnd_abs,blue_cumsum_sig_gnd_seen,red_cumsum_sig_gnd_seen,blue_cumsum_fight_sec,red_cumsum_fight_sec,blue_cumsum_decWin,red_cumsum_decWin,blue_cumsum_subWin,red_cumsum_subWin,blue_cumsum_KOWin,red_cumsum_KOWin,blue_cumsum_Decloss,red_cumsum_Decloss,blue_cumsum_subLoss,red_cumsum_subLoss,blue_cumsum_KOLoss,red_cumsum_KOLoss,blue_wins,red_wins,blue_losses,red_losses,blue_titleFights,red_titleFights,red_total_stk_per_min,red_sig_stk_per_min,red_total_stk_acc,red_sig_stk_acc,red_stk_head_per_min,red_stk_body_per_min,red_stk_leg_per_min,red_stk_dist_per_min,red_stk_clch_per_min,red_stk_gnd_per_min,red_total_stk_abs_per_min,red_sig_stk_abs_per_min,red_stk_head_abs_per_min,red_stk_body_abs_per_min,red_stk_leg_abs_per_min,red_stk_dist_abs_per_min,red_stk_clch_abs_per_min,red_stk_gnd_abs_per_min,red_total_stk_per_min_diff,red_sig_stk_per_min_diff,red_ctrl_sec_per_min,red_ctrld_sec_per_min,red_ctrl_diff,red_TD_acc,red_TD_def,red_TD_per_min,red_TD_abs_per_min,red_KDs_per_100_stk,red_reach_dif,red_height_dif,blue_total_stk_per_min,blue_sig_stk_per_min,blue_total_stk_acc,blue_sig_stk_acc,blue_stk_head_per_min,blue_stk_body_per_min,blue_stk_leg_per_min,blue_stk_dist_per_min,blue_stk_clch_per_min,blue_stk_gnd_per_min,blue_total_stk_abs_per_min,blue_sig_stk_abs_per_min,blue_stk_head_abs_per_min,blue_stk_body_abs_per_min,blue_stk_leg_abs_per_min,blue_stk_dist_abs_per_min,blue_stk_clch_abs_per_min,blue_stk_gnd_abs_per_min,blue_total_stk_per_min_diff,blue_sig_stk_per_min_diff,blue_ctrl_sec_per_min,blue_ctrld_sec_per_min,blue_ctrl_diff,blue_TD_acc,blue_TD_def,blue_KDs_per_100_stk,blue_reach_dif,blue_height_dif,winner,weight_class,title_fight
0,0,30.0,37.0,69,71,Kelvin Gastelum,Jared Cannonier,71,77,Southpaw,Switch,1100,433,2326,763,842,357,2006,681,19,1,51,3,2,0,2251,364,7,4,7,0,165,40,251,63,576,247,1534,640,136,53,162,64,696,253,1686,631,121,39,187,58,60,48,74,78,1018,422,2117,1001,787,299,1850,837,29,17

In [3]:
# isolating our future fights to predict
future_df = big_df[big_df["winner"] == "tbd"]
future_df

,lev1,blue_age,red_age,blue_heightIN,red_heightIN,blue_name,red_name,blue_reach,red_reach,blue_stance,red_stance,blue_cumsum_tot_stks_lnd,red_cumsum_tot_stks_lnd,blue_cumsum_tot_stks_att,red_cumsum_tot_stks_att,blue_cumsum_sig_stks_lnd,red_cumsum_sig_stks_lnd,blue_cumsum_sig_stks_att,red_cumsum_sig_stks_att,blue_cumsum_TDs_lnd,red_cumsum_TDs_lnd,blue_cumsum_TDs_att,red_cumsum_TDs_att,blue_cumsum_sub_att,red_cumsum_sub_att,blue_cumsum_ctrl_sec,red_cumsum_ctrl_sec,blue_cumsum_KDs,red_cumsum_KDs,blue_cumsum_revs,red_cumsum_revs,blue_cumsum_sig_body_lnd,red_cumsum_sig_body_lnd,blue_cumsum_sig_body_att,red_cumsum_sig_body_att,blue_cumsum_sig_head_lnd,red_cumsum_sig_head_lnd,blue_cumsum_sig_head_att,red_cumsum_sig_head_att,blue_cumsum_sig_leg_lnd,red_cumsum_sig_leg_lnd,blue_cumsum_sig_leg_att,red_cumsum_sig_leg_att,blue_cumsum_sig_dist_lnd,red_cumsum_sig_dist_lnd,blue_cumsum_sig_dist_att,red_cumsum_sig_dist_att,blue_cumsum_sig_clch_lnd,red_cumsum_sig_clch_lnd,blue_cumsum_sig_clch_att,red_cumsum_sig_clch_att,blue_cumsum_sig_gnd_lnd,red_cumsum_sig_gnd_lnd,blue_cumsum_sig_gnd_att,red_cumsum_sig_gnd_att,blue_cumsum_tot_stks_abs,red_cumsum_tot_stks_abs,blue_cumsum_tot_stks_seen,red_cumsum_tot_stks_seen,blue_cumsum_sig_stks_abs,red_cumsum_sig_stks_abs,blue_cumsum_sig_stks_seen,red_cumsum_sig_stks_seen,blue_cumsum_TDs_abs,red_cumsum_TDs_abs,blue_cumsum_TDs_seen,red_cumsum_TDs_seen,blue_cumsum_sub_esc,red_cumsum_sub_esc,blue_cumsum_ctrld_sec,red_cumsum_ctrld_sec,blue_cumsum_KedD,red_cumsum_KedD,blue_cumsum_reversed,red_cumsum_reversed,blue_cumsum_sig_body_abs,red_cumsum_sig_body_abs,blue_cumsum_sig_body_seen,red_cumsum_sig_body_seen,blue_cumsum_sig_head_abs,red_cumsum_sig_head_abs,blue_cumsum_sig_head_seen,red_cumsum_sig_head_seen,blue_cumsum_sig_leg_abs,red_cumsum_sig_leg_abs,blue_cumsum_sig_leg_seen,red_cumsum_sig_leg_seen,blue_cumsum_sig_dist_abs,red_cumsum_sig_dist_abs,blue_cumsum_sig_dist_seen,red_cumsum_sig_dist_seen,blue_cumsum_sig_clch_abs,red_cumsum_sig_clch_abs,blue_cumsum_sig_clch_seen,red_cumsum_sig_clch_seen,blue_cumsum_sig_gnd_abs,red_cumsum_sig_gnd_abs,blue_cumsum_sig_gnd_seen,red_cumsum_sig_gnd_seen,blue_cumsum_fight_sec,red_cumsum_fight_sec,blue_cumsum_decWin,red_cumsum_decWin,blue_cumsum_subWin,red_cumsum_subWin,blue_cumsum_KOWin,red_cumsum_KOWin,blue_cumsum_Decloss,red_cumsum_Decloss,blue_cumsum_subLoss,red_cumsum_subLoss,blue_cumsum_KOLoss,red_cumsum_KOLoss,blue_wins,red_wins,blue_losses,red_losses,blue_titleFights,red_titleFights,red_total_stk_per_min,red_sig_stk_per_min,red_total_stk_acc,red_sig_stk_acc,red_stk_head_per_min,red_stk_body_per_min,red_stk_leg_per_min,red_stk_dist_per_min,red_stk_clch_per_min,red_stk_gnd_per_min,red_total_stk_abs_per_min,red_sig_stk_abs_per_min,red_stk_head_abs_per_min,red_stk_body_abs_per_min,red_stk_leg_abs_per_min,red_stk_dist_abs_per_min,red_stk_clch_abs_per_min,red_stk_gnd_abs_per_min,red_total_stk_per_min_diff,red_sig_stk_per_min_diff,red_ctrl_sec_per_min,red_ctrld_sec_per_min,red_ctrl_diff,red_TD_acc,red_TD_def,red_TD_per_min,red_TD_abs_per_min,red_KDs_per_100_stk,red_reach_dif,red_height_dif,blue_total_stk_per_min,blue_sig_stk_per_min,blue_total_stk_acc,blue_sig_stk_acc,blue_stk_head_per_min,blue_stk_body_per_min,blue_stk_leg_per_min,blue_stk_dist_per_min,blue_stk_clch_per_min,blue_stk_gnd_per_min,blue_total_stk_abs_per_min,blue_sig_stk_abs_per_min,blue_stk_head_abs_per_min,blue_stk_body_abs_per_min,blue_stk_leg_abs_per_min,blue_stk_dist_abs_per_min,blue_stk_clch_abs_per_min,blue_stk_gnd_abs_per_min,blue_total_stk_per_min_diff,blue_sig_stk_per_min_diff,blue_ctrl_sec_per_min,blue_ctrld_sec_per_min,blue_ctrl_diff,blue_TD_acc,blue_TD_def,blue_KDs_per_100_stk,blue_reach_dif,blue_height_dif,winner,weight_class,title_fight
0,0,30.0,37.0,69,71,Kelvin Gastelum,Jared Cannonier,71,77,Southpaw,Switch,1100,433,2326,763,842,357,2006,681,19,1,51,3,2,0,2251,364,7,4,7,0,165,40,251,63,576,247,1534,640,136,53,162,64,696,253,1686,631,121,39,187,58,60,48,74,78,1018,422,2117,1001,787,299,1850,837,29,17

In [4]:
# Label encode our stances and weight class 
number = LabelEncoder()
# data['red_name'] = number.fit_transform(data['red_name'].astype('str'))
# data['blue_name'] = number.fit_transform(data['blue_name'].astype('str'))
future_df['red_stance'] = number.fit_transform(future_df['red_stance'].astype('str'))
future_df['blue_stance'] = number.fit_transform(future_df['blue_stance'].astype('str'))
future_df['weight_class'] = number.fit_transform(future_df['weight_class'].astype('str'))
future_df

C:\Users\audri\anaconda3\envs\PythonAdv\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
C:\Users\audri\anaconda3\envs\PythonAdv\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\audri\anaconda3\envs\PythonAdv\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats i

,lev1,blue_age,red_age,blue_heightIN,red_heightIN,blue_name,red_name,blue_reach,red_reach,blue_stance,red_stance,blue_cumsum_tot_stks_lnd,red_cumsum_tot_stks_lnd,blue_cumsum_tot_stks_att,red_cumsum_tot_stks_att,blue_cumsum_sig_stks_lnd,red_cumsum_sig_stks_lnd,blue_cumsum_sig_stks_att,red_cumsum_sig_stks_att,blue_cumsum_TDs_lnd,red_cumsum_TDs_lnd,blue_cumsum_TDs_att,red_cumsum_TDs_att,blue_cumsum_sub_att,red_cumsum_sub_att,blue_cumsum_ctrl_sec,red_cumsum_ctrl_sec,blue_cumsum_KDs,red_cumsum_KDs,blue_cumsum_revs,red_cumsum_revs,blue_cumsum_sig_body_lnd,red_cumsum_sig_body_lnd,blue_cumsum_sig_body_att,red_cumsum_sig_body_att,blue_cumsum_sig_head_lnd,red_cumsum_sig_head_lnd,blue_cumsum_sig_head_att,red_cumsum_sig_head_att,blue_cumsum_sig_leg_lnd,red_cumsum_sig_leg_lnd,blue_cumsum_sig_leg_att,red_cumsum_sig_leg_att,blue_cumsum_sig_dist_lnd,red_cumsum_sig_dist_lnd,blue_cumsum_sig_dist_att,red_cumsum_sig_dist_att,blue_cumsum_sig_clch_lnd,red_cumsum_sig_clch_lnd,blue_cumsum_sig_clch_att,red_cumsum_sig_clch_att,blue_cumsum_sig_gnd_lnd,red_cumsum_sig_gnd_lnd,blue_cumsum_sig_gnd_att,red_cumsum_sig_gnd_att,blue_cumsum_tot_stks_abs,red_cumsum_tot_stks_abs,blue_cumsum_tot_stks_seen,red_cumsum_tot_stks_seen,blue_cumsum_sig_stks_abs,red_cumsum_sig_stks_abs,blue_cumsum_sig_stks_seen,red_cumsum_sig_stks_seen,blue_cumsum_TDs_abs,red_cumsum_TDs_abs,blue_cumsum_TDs_seen,red_cumsum_TDs_seen,blue_cumsum_sub_esc,red_cumsum_sub_esc,blue_cumsum_ctrld_sec,red_cumsum_ctrld_sec,blue_cumsum_KedD,red_cumsum_KedD,blue_cumsum_reversed,red_cumsum_reversed,blue_cumsum_sig_body_abs,red_cumsum_sig_body_abs,blue_cumsum_sig_body_seen,red_cumsum_sig_body_seen,blue_cumsum_sig_head_abs,red_cumsum_sig_head_abs,blue_cumsum_sig_head_seen,red_cumsum_sig_head_seen,blue_cumsum_sig_leg_abs,red_cumsum_sig_leg_abs,blue_cumsum_sig_leg_seen,red_cumsum_sig_leg_seen,blue_cumsum_sig_dist_abs,red_cumsum_sig_dist_abs,blue_cumsum_sig_dist_seen,red_cumsum_sig_dist_seen,blue_cumsum_sig_clch_abs,red_cumsum_sig_clch_abs,blue_cumsum_sig_clch_seen,red_cumsum_sig_clch_seen,blue_cumsum_sig_gnd_abs,red_cumsum_sig_gnd_abs,blue_cumsum_sig_gnd_seen,red_cumsum_sig_gnd_seen,blue_cumsum_fight_sec,red_cumsum_fight_sec,blue_cumsum_decWin,red_cumsum_decWin,blue_cumsum_subWin,red_cumsum_subWin,blue_cumsum_KOWin,red_cumsum_KOWin,blue_cumsum_Decloss,red_cumsum_Decloss,blue_cumsum_subLoss,red_cumsum_subLoss,blue_cumsum_KOLoss,red_cumsum_KOLoss,blue_wins,red_wins,blue_losses,red_losses,blue_titleFights,red_titleFights,red_total_stk_per_min,red_sig_stk_per_min,red_total_stk_acc,red_sig_stk_acc,red_stk_head_per_min,red_stk_body_per_min,red_stk_leg_per_min,red_stk_dist_per_min,red_stk_clch_per_min,red_stk_gnd_per_min,red_total_stk_abs_per_min,red_sig_stk_abs_per_min,red_stk_head_abs_per_min,red_stk_body_abs_per_min,red_stk_leg_abs_per_min,red_stk_dist_abs_per_min,red_stk_clch_abs_per_min,red_stk_gnd_abs_per_min,red_total_stk_per_min_diff,red_sig_stk_per_min_diff,red_ctrl_sec_per_min,red_ctrld_sec_per_min,red_ctrl_diff,red_TD_acc,red_TD_def,red_TD_per_min,red_TD_abs_per_min,red_KDs_per_100_stk,red_reach_dif,red_height_dif,blue_total_stk_per_min,blue_sig_stk_per_min,blue_total_stk_acc,blue_sig_stk_acc,blue_stk_head_per_min,blue_stk_body_per_min,blue_stk_leg_per_min,blue_stk_dist_per_min,blue_stk_clch_per_min,blue_stk_gnd_per_min,blue_total_stk_abs_per_min,blue_sig_stk_abs_per_min,blue_stk_head_abs_per_min,blue_stk_body_abs_per_min,blue_stk_leg_abs_per_min,blue_stk_dist_abs_per_min,blue_stk_clch_abs_per_min,blue_stk_gnd_abs_per_min,blue_total_stk_per_min_diff,blue_sig_stk_per_min_diff,blue_ctrl_sec_per_min,blue_ctrld_sec_per_min,blue_ctrl_diff,blue_TD_acc,blue_TD_def,blue_KDs_per_100_stk,blue_reach_dif,blue_height_dif,winner,weight_class,title_fight
0,0,30.0,37.0,69,71,Kelvin Gastelum,Jared Cannonier,71,77,1,1,1100,433,2326,763,842,357,2006,681,19,1,51,3,2,0,2251,364,7,4,7,0,165,40,251,63,576,247,1534,640,136,53,162,64,696,253,1686,631,121,39,187,58,60,48,74,78,1018,422,2117,1001,787,299,1850,837,29,17,78,37,8,1,2

In [5]:
# drop columns
data = future_df.drop(columns=["lev1", "winner", "red_name", "blue_name"])
# target = train_test_df["winner"]
# feature_names = data.columns

In [6]:
# load in our saved random forest model...
loaded_model = pickle.load(open("../saved_models/rf_up_model.sav", 'rb'))
pred = loaded_model.predict(data)
future_df["prediction"] = pred
future_df

C:\Users\audri\anaconda3\envs\PythonAdv\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


,lev1,blue_age,red_age,blue_heightIN,red_heightIN,blue_name,red_name,blue_reach,red_reach,blue_stance,red_stance,blue_cumsum_tot_stks_lnd,red_cumsum_tot_stks_lnd,blue_cumsum_tot_stks_att,red_cumsum_tot_stks_att,blue_cumsum_sig_stks_lnd,red_cumsum_sig_stks_lnd,blue_cumsum_sig_stks_att,red_cumsum_sig_stks_att,blue_cumsum_TDs_lnd,red_cumsum_TDs_lnd,blue_cumsum_TDs_att,red_cumsum_TDs_att,blue_cumsum_sub_att,red_cumsum_sub_att,blue_cumsum_ctrl_sec,red_cumsum_ctrl_sec,blue_cumsum_KDs,red_cumsum_KDs,blue_cumsum_revs,red_cumsum_revs,blue_cumsum_sig_body_lnd,red_cumsum_sig_body_lnd,blue_cumsum_sig_body_att,red_cumsum_sig_body_att,blue_cumsum_sig_head_lnd,red_cumsum_sig_head_lnd,blue_cumsum_sig_head_att,red_cumsum_sig_head_att,blue_cumsum_sig_leg_lnd,red_cumsum_sig_leg_lnd,blue_cumsum_sig_leg_att,red_cumsum_sig_leg_att,blue_cumsum_sig_dist_lnd,red_cumsum_sig_dist_lnd,blue_cumsum_sig_dist_att,red_cumsum_sig_dist_att,blue_cumsum_sig_clch_lnd,red_cumsum_sig_clch_lnd,blue_cumsum_sig_clch_att,red_cumsum_sig_clch_att,blue_cumsum_sig_gnd_lnd,red_cumsum_sig_gnd_lnd,blue_cumsum_sig_gnd_att,red_cumsum_sig_gnd_att,blue_cumsum_tot_stks_abs,red_cumsum_tot_stks_abs,blue_cumsum_tot_stks_seen,red_cumsum_tot_stks_seen,blue_cumsum_sig_stks_abs,red_cumsum_sig_stks_abs,blue_cumsum_sig_stks_seen,red_cumsum_sig_stks_seen,blue_cumsum_TDs_abs,red_cumsum_TDs_abs,blue_cumsum_TDs_seen,red_cumsum_TDs_seen,blue_cumsum_sub_esc,red_cumsum_sub_esc,blue_cumsum_ctrld_sec,red_cumsum_ctrld_sec,blue_cumsum_KedD,red_cumsum_KedD,blue_cumsum_reversed,red_cumsum_reversed,blue_cumsum_sig_body_abs,red_cumsum_sig_body_abs,blue_cumsum_sig_body_seen,red_cumsum_sig_body_seen,blue_cumsum_sig_head_abs,red_cumsum_sig_head_abs,blue_cumsum_sig_head_seen,red_cumsum_sig_head_seen,blue_cumsum_sig_leg_abs,red_cumsum_sig_leg_abs,blue_cumsum_sig_leg_seen,red_cumsum_sig_leg_seen,blue_cumsum_sig_dist_abs,red_cumsum_sig_dist_abs,blue_cumsum_sig_dist_seen,red_cumsum_sig_dist_seen,blue_cumsum_sig_clch_abs,red_cumsum_sig_clch_abs,blue_cumsum_sig_clch_seen,red_cumsum_sig_clch_seen,blue_cumsum_sig_gnd_abs,red_cumsum_sig_gnd_abs,blue_cumsum_sig_gnd_seen,red_cumsum_sig_gnd_seen,blue_cumsum_fight_sec,red_cumsum_fight_sec,blue_cumsum_decWin,red_cumsum_decWin,blue_cumsum_subWin,red_cumsum_subWin,blue_cumsum_KOWin,red_cumsum_KOWin,blue_cumsum_Decloss,red_cumsum_Decloss,blue_cumsum_subLoss,red_cumsum_subLoss,blue_cumsum_KOLoss,red_cumsum_KOLoss,blue_wins,red_wins,blue_losses,red_losses,blue_titleFights,red_titleFights,red_total_stk_per_min,red_sig_stk_per_min,red_total_stk_acc,red_sig_stk_acc,red_stk_head_per_min,red_stk_body_per_min,red_stk_leg_per_min,red_stk_dist_per_min,red_stk_clch_per_min,red_stk_gnd_per_min,red_total_stk_abs_per_min,red_sig_stk_abs_per_min,red_stk_head_abs_per_min,red_stk_body_abs_per_min,red_stk_leg_abs_per_min,red_stk_dist_abs_per_min,red_stk_clch_abs_per_min,red_stk_gnd_abs_per_min,red_total_stk_per_min_diff,red_sig_stk_per_min_diff,red_ctrl_sec_per_min,red_ctrld_sec_per_min,red_ctrl_diff,red_TD_acc,red_TD_def,red_TD_per_min,red_TD_abs_per_min,red_KDs_per_100_stk,red_reach_dif,red_height_dif,blue_total_stk_per_min,blue_sig_stk_per_min,blue_total_stk_acc,blue_sig_stk_acc,blue_stk_head_per_min,blue_stk_body_per_min,blue_stk_leg_per_min,blue_stk_dist_per_min,blue_stk_clch_per_min,blue_stk_gnd_per_min,blue_total_stk_abs_per_min,blue_sig_stk_abs_per_min,blue_stk_head_abs_per_min,blue_stk_body_abs_per_min,blue_stk_leg_abs_per_min,blue_stk_dist_abs_per_min,blue_stk_clch_abs_per_min,blue_stk_gnd_abs_per_min,blue_total_stk_per_min_diff,blue_sig_stk_per_min_diff,blue_ctrl_sec_per_min,blue_ctrld_sec_per_min,blue_ctrl_diff,blue_TD_acc,blue_TD_def,blue_KDs_per_100_stk,blue_reach_dif,blue_height_dif,winner,weight_class,title_fight,prediction
0,0,30.0,37.0,69,71,Kelvin Gastelum,Jared Cannonier,71,77,1,1,1100,433,2326,763,842,357,2006,681,19,1,51,3,2,0,2251,364,7,4,7,0,165,40,251,63,576,247,1534,640,136,53,162,64,696,253,1686,631,121,39,187,58,60,48,74,78,1018,422,2117,1001,787,299,1850,837,29,17,

In [7]:
# Getting percentage certainty of prediction as an array of two outcomes, totaling 100
pcts = loaded_model.predict_proba(data)

# Looping through to get our favorite and dog in each fight
favs = []
dogs = []
for pct in pcts:
    fav = max(pct)
    dog = min(pct)
    favs.append(fav)
    dogs.append(dog)

In [8]:
# and adding these percentages to the dataframe
future_df["fav_pct"] = favs
future_df["dog_pct"] = dogs
future_df

C:\Users\audri\anaconda3\envs\PythonAdv\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\audri\anaconda3\envs\PythonAdv\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,lev1,blue_age,red_age,blue_heightIN,red_heightIN,blue_name,red_name,blue_reach,red_reach,blue_stance,red_stance,blue_cumsum_tot_stks_lnd,red_cumsum_tot_stks_lnd,blue_cumsum_tot_stks_att,red_cumsum_tot_stks_att,blue_cumsum_sig_stks_lnd,red_cumsum_sig_stks_lnd,blue_cumsum_sig_stks_att,red_cumsum_sig_stks_att,blue_cumsum_TDs_lnd,red_cumsum_TDs_lnd,blue_cumsum_TDs_att,red_cumsum_TDs_att,blue_cumsum_sub_att,red_cumsum_sub_att,blue_cumsum_ctrl_sec,red_cumsum_ctrl_sec,blue_cumsum_KDs,red_cumsum_KDs,blue_cumsum_revs,red_cumsum_revs,blue_cumsum_sig_body_lnd,red_cumsum_sig_body_lnd,blue_cumsum_sig_body_att,red_cumsum_sig_body_att,blue_cumsum_sig_head_lnd,red_cumsum_sig_head_lnd,blue_cumsum_sig_head_att,red_cumsum_sig_head_att,blue_cumsum_sig_leg_lnd,red_cumsum_sig_leg_lnd,blue_cumsum_sig_leg_att,red_cumsum_sig_leg_att,blue_cumsum_sig_dist_lnd,red_cumsum_sig_dist_lnd,blue_cumsum_sig_dist_att,red_cumsum_sig_dist_att,blue_cumsum_sig_clch_lnd,red_cumsum_sig_clch_lnd,blue_cumsum_sig_clch_att,red_cumsum_sig_clch_att,blue_cumsum_sig_gnd_lnd,red_cumsum_sig_gnd_lnd,blue_cumsum_sig_gnd_att,red_cumsum_sig_gnd_att,blue_cumsum_tot_stks_abs,red_cumsum_tot_stks_abs,blue_cumsum_tot_stks_seen,red_cumsum_tot_stks_seen,blue_cumsum_sig_stks_abs,red_cumsum_sig_stks_abs,blue_cumsum_sig_stks_seen,red_cumsum_sig_stks_seen,blue_cumsum_TDs_abs,red_cumsum_TDs_abs,blue_cumsum_TDs_seen,red_cumsum_TDs_seen,blue_cumsum_sub_esc,red_cumsum_sub_esc,blue_cumsum_ctrld_sec,red_cumsum_ctrld_sec,blue_cumsum_KedD,red_cumsum_KedD,blue_cumsum_reversed,red_cumsum_reversed,blue_cumsum_sig_body_abs,red_cumsum_sig_body_abs,blue_cumsum_sig_body_seen,red_cumsum_sig_body_seen,blue_cumsum_sig_head_abs,red_cumsum_sig_head_abs,blue_cumsum_sig_head_seen,red_cumsum_sig_head_seen,blue_cumsum_sig_leg_abs,red_cumsum_sig_leg_abs,blue_cumsum_sig_leg_seen,red_cumsum_sig_leg_seen,blue_cumsum_sig_dist_abs,red_cumsum_sig_dist_abs,blue_cumsum_sig_dist_seen,red_cumsum_sig_dist_seen,blue_cumsum_sig_clch_abs,red_cumsum_sig_clch_abs,blue_cumsum_sig_clch_seen,red_cumsum_sig_clch_seen,blue_cumsum_sig_gnd_abs,red_cumsum_sig_gnd_abs,blue_cumsum_sig_gnd_seen,red_cumsum_sig_gnd_seen,blue_cumsum_fight_sec,red_cumsum_fight_sec,blue_cumsum_decWin,red_cumsum_decWin,blue_cumsum_subWin,red_cumsum_subWin,blue_cumsum_KOWin,red_cumsum_KOWin,blue_cumsum_Decloss,red_cumsum_Decloss,blue_cumsum_subLoss,red_cumsum_subLoss,blue_cumsum_KOLoss,red_cumsum_KOLoss,blue_wins,red_wins,blue_losses,red_losses,blue_titleFights,red_titleFights,red_total_stk_per_min,red_sig_stk_per_min,red_total_stk_acc,red_sig_stk_acc,red_stk_head_per_min,red_stk_body_per_min,red_stk_leg_per_min,red_stk_dist_per_min,red_stk_clch_per_min,red_stk_gnd_per_min,red_total_stk_abs_per_min,red_sig_stk_abs_per_min,red_stk_head_abs_per_min,red_stk_body_abs_per_min,red_stk_leg_abs_per_min,red_stk_dist_abs_per_min,red_stk_clch_abs_per_min,red_stk_gnd_abs_per_min,red_total_stk_per_min_diff,red_sig_stk_per_min_diff,red_ctrl_sec_per_min,red_ctrld_sec_per_min,red_ctrl_diff,red_TD_acc,red_TD_def,red_TD_per_min,red_TD_abs_per_min,red_KDs_per_100_stk,red_reach_dif,red_height_dif,blue_total_stk_per_min,blue_sig_stk_per_min,blue_total_stk_acc,blue_sig_stk_acc,blue_stk_head_per_min,blue_stk_body_per_min,blue_stk_leg_per_min,blue_stk_dist_per_min,blue_stk_clch_per_min,blue_stk_gnd_per_min,blue_total_stk_abs_per_min,blue_sig_stk_abs_per_min,blue_stk_head_abs_per_min,blue_stk_body_abs_per_min,blue_stk_leg_abs_per_min,blue_stk_dist_abs_per_min,blue_stk_clch_abs_per_min,blue_stk_gnd_abs_per_min,blue_total_stk_per_min_diff,blue_sig_stk_per_min_diff,blue_ctrl_sec_per_min,blue_ctrld_sec_per_min,blue_ctrl_diff,blue_TD_acc,blue_TD_def,blue_KDs_per_100_stk,blue_reach_dif,blue_height_dif,winner,weight_class,title_fight,prediction,fav_pct,dog_pct
0,0,30.0,37.0,69,71,Kelvin Gastelum,Jared Cannonier,71,77,1,1,1100,433,2326,763,842,357,2006,681,19,1,51,3,2,0,2251,364,7,4,7,0,165,40,251,63,576,247,1534,640,136,53,162,64,696,253,1686,631,121,39,187,58,60,48,74,78,1018,422,2117,1001,787,299

In [9]:
# round off percentages to make for cleaner viewing
future_df["fav_pct"] = future_df["fav_pct"].round(2)
future_df["dog_pct"] = future_df["dog_pct"].round(2)
future_df

C:\Users\audri\anaconda3\envs\PythonAdv\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\audri\anaconda3\envs\PythonAdv\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,lev1,blue_age,red_age,blue_heightIN,red_heightIN,blue_name,red_name,blue_reach,red_reach,blue_stance,red_stance,blue_cumsum_tot_stks_lnd,red_cumsum_tot_stks_lnd,blue_cumsum_tot_stks_att,red_cumsum_tot_stks_att,blue_cumsum_sig_stks_lnd,red_cumsum_sig_stks_lnd,blue_cumsum_sig_stks_att,red_cumsum_sig_stks_att,blue_cumsum_TDs_lnd,red_cumsum_TDs_lnd,blue_cumsum_TDs_att,red_cumsum_TDs_att,blue_cumsum_sub_att,red_cumsum_sub_att,blue_cumsum_ctrl_sec,red_cumsum_ctrl_sec,blue_cumsum_KDs,red_cumsum_KDs,blue_cumsum_revs,red_cumsum_revs,blue_cumsum_sig_body_lnd,red_cumsum_sig_body_lnd,blue_cumsum_sig_body_att,red_cumsum_sig_body_att,blue_cumsum_sig_head_lnd,red_cumsum_sig_head_lnd,blue_cumsum_sig_head_att,red_cumsum_sig_head_att,blue_cumsum_sig_leg_lnd,red_cumsum_sig_leg_lnd,blue_cumsum_sig_leg_att,red_cumsum_sig_leg_att,blue_cumsum_sig_dist_lnd,red_cumsum_sig_dist_lnd,blue_cumsum_sig_dist_att,red_cumsum_sig_dist_att,blue_cumsum_sig_clch_lnd,red_cumsum_sig_clch_lnd,blue_cumsum_sig_clch_att,red_cumsum_sig_clch_att,blue_cumsum_sig_gnd_lnd,red_cumsum_sig_gnd_lnd,blue_cumsum_sig_gnd_att,red_cumsum_sig_gnd_att,blue_cumsum_tot_stks_abs,red_cumsum_tot_stks_abs,blue_cumsum_tot_stks_seen,red_cumsum_tot_stks_seen,blue_cumsum_sig_stks_abs,red_cumsum_sig_stks_abs,blue_cumsum_sig_stks_seen,red_cumsum_sig_stks_seen,blue_cumsum_TDs_abs,red_cumsum_TDs_abs,blue_cumsum_TDs_seen,red_cumsum_TDs_seen,blue_cumsum_sub_esc,red_cumsum_sub_esc,blue_cumsum_ctrld_sec,red_cumsum_ctrld_sec,blue_cumsum_KedD,red_cumsum_KedD,blue_cumsum_reversed,red_cumsum_reversed,blue_cumsum_sig_body_abs,red_cumsum_sig_body_abs,blue_cumsum_sig_body_seen,red_cumsum_sig_body_seen,blue_cumsum_sig_head_abs,red_cumsum_sig_head_abs,blue_cumsum_sig_head_seen,red_cumsum_sig_head_seen,blue_cumsum_sig_leg_abs,red_cumsum_sig_leg_abs,blue_cumsum_sig_leg_seen,red_cumsum_sig_leg_seen,blue_cumsum_sig_dist_abs,red_cumsum_sig_dist_abs,blue_cumsum_sig_dist_seen,red_cumsum_sig_dist_seen,blue_cumsum_sig_clch_abs,red_cumsum_sig_clch_abs,blue_cumsum_sig_clch_seen,red_cumsum_sig_clch_seen,blue_cumsum_sig_gnd_abs,red_cumsum_sig_gnd_abs,blue_cumsum_sig_gnd_seen,red_cumsum_sig_gnd_seen,blue_cumsum_fight_sec,red_cumsum_fight_sec,blue_cumsum_decWin,red_cumsum_decWin,blue_cumsum_subWin,red_cumsum_subWin,blue_cumsum_KOWin,red_cumsum_KOWin,blue_cumsum_Decloss,red_cumsum_Decloss,blue_cumsum_subLoss,red_cumsum_subLoss,blue_cumsum_KOLoss,red_cumsum_KOLoss,blue_wins,red_wins,blue_losses,red_losses,blue_titleFights,red_titleFights,red_total_stk_per_min,red_sig_stk_per_min,red_total_stk_acc,red_sig_stk_acc,red_stk_head_per_min,red_stk_body_per_min,red_stk_leg_per_min,red_stk_dist_per_min,red_stk_clch_per_min,red_stk_gnd_per_min,red_total_stk_abs_per_min,red_sig_stk_abs_per_min,red_stk_head_abs_per_min,red_stk_body_abs_per_min,red_stk_leg_abs_per_min,red_stk_dist_abs_per_min,red_stk_clch_abs_per_min,red_stk_gnd_abs_per_min,red_total_stk_per_min_diff,red_sig_stk_per_min_diff,red_ctrl_sec_per_min,red_ctrld_sec_per_min,red_ctrl_diff,red_TD_acc,red_TD_def,red_TD_per_min,red_TD_abs_per_min,red_KDs_per_100_stk,red_reach_dif,red_height_dif,blue_total_stk_per_min,blue_sig_stk_per_min,blue_total_stk_acc,blue_sig_stk_acc,blue_stk_head_per_min,blue_stk_body_per_min,blue_stk_leg_per_min,blue_stk_dist_per_min,blue_stk_clch_per_min,blue_stk_gnd_per_min,blue_total_stk_abs_per_min,blue_sig_stk_abs_per_min,blue_stk_head_abs_per_min,blue_stk_body_abs_per_min,blue_stk_leg_abs_per_min,blue_stk_dist_abs_per_min,blue_stk_clch_abs_per_min,blue_stk_gnd_abs_per_min,blue_total_stk_per_min_diff,blue_sig_stk_per_min_diff,blue_ctrl_sec_per_min,blue_ctrld_sec_per_min,blue_ctrl_diff,blue_TD_acc,blue_TD_def,blue_KDs_per_100_stk,blue_reach_dif,blue_height_dif,winner,weight_class,title_fight,prediction,fav_pct,dog_pct
0,0,30.0,37.0,69,71,Kelvin Gastelum,Jared Cannonier,71,77,1,1,1100,433,2326,763,842,357,2006,681,19,1,51,3,2,0,2251,364,7,4,7,0,165,40,251,63,576,247,1534,640,136,53,162,64,696,253,1686,631,121,39,187,58,60,48,74,78,1018,422,2117,1001,787,299

In [10]:
# Converting percentages to a moneyline as would be seen at a fight bookie, without the house weighting
future_df["fav_moneyline"] = (future_df["fav_pct"] * 100) / (100 - (future_df["fav_pct"] * 100)) * (-100)
future_df["dog_moneyline"] = ((100 - (future_df["dog_pct"] * 100)) / (future_df["dog_pct"] * 100) * 100)
future_df

C:\Users\audri\anaconda3\envs\PythonAdv\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\audri\anaconda3\envs\PythonAdv\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,lev1,blue_age,red_age,blue_heightIN,red_heightIN,blue_name,red_name,blue_reach,red_reach,blue_stance,red_stance,blue_cumsum_tot_stks_lnd,red_cumsum_tot_stks_lnd,blue_cumsum_tot_stks_att,red_cumsum_tot_stks_att,blue_cumsum_sig_stks_lnd,red_cumsum_sig_stks_lnd,blue_cumsum_sig_stks_att,red_cumsum_sig_stks_att,blue_cumsum_TDs_lnd,red_cumsum_TDs_lnd,blue_cumsum_TDs_att,red_cumsum_TDs_att,blue_cumsum_sub_att,red_cumsum_sub_att,blue_cumsum_ctrl_sec,red_cumsum_ctrl_sec,blue_cumsum_KDs,red_cumsum_KDs,blue_cumsum_revs,red_cumsum_revs,blue_cumsum_sig_body_lnd,red_cumsum_sig_body_lnd,blue_cumsum_sig_body_att,red_cumsum_sig_body_att,blue_cumsum_sig_head_lnd,red_cumsum_sig_head_lnd,blue_cumsum_sig_head_att,red_cumsum_sig_head_att,blue_cumsum_sig_leg_lnd,red_cumsum_sig_leg_lnd,blue_cumsum_sig_leg_att,red_cumsum_sig_leg_att,blue_cumsum_sig_dist_lnd,red_cumsum_sig_dist_lnd,blue_cumsum_sig_dist_att,red_cumsum_sig_dist_att,blue_cumsum_sig_clch_lnd,red_cumsum_sig_clch_lnd,blue_cumsum_sig_clch_att,red_cumsum_sig_clch_att,blue_cumsum_sig_gnd_lnd,red_cumsum_sig_gnd_lnd,blue_cumsum_sig_gnd_att,red_cumsum_sig_gnd_att,blue_cumsum_tot_stks_abs,red_cumsum_tot_stks_abs,blue_cumsum_tot_stks_seen,red_cumsum_tot_stks_seen,blue_cumsum_sig_stks_abs,red_cumsum_sig_stks_abs,blue_cumsum_sig_stks_seen,red_cumsum_sig_stks_seen,blue_cumsum_TDs_abs,red_cumsum_TDs_abs,blue_cumsum_TDs_seen,red_cumsum_TDs_seen,blue_cumsum_sub_esc,red_cumsum_sub_esc,blue_cumsum_ctrld_sec,red_cumsum_ctrld_sec,blue_cumsum_KedD,red_cumsum_KedD,blue_cumsum_reversed,red_cumsum_reversed,blue_cumsum_sig_body_abs,red_cumsum_sig_body_abs,blue_cumsum_sig_body_seen,red_cumsum_sig_body_seen,blue_cumsum_sig_head_abs,red_cumsum_sig_head_abs,blue_cumsum_sig_head_seen,red_cumsum_sig_head_seen,blue_cumsum_sig_leg_abs,red_cumsum_sig_leg_abs,blue_cumsum_sig_leg_seen,red_cumsum_sig_leg_seen,blue_cumsum_sig_dist_abs,red_cumsum_sig_dist_abs,blue_cumsum_sig_dist_seen,red_cumsum_sig_dist_seen,blue_cumsum_sig_clch_abs,red_cumsum_sig_clch_abs,blue_cumsum_sig_clch_seen,red_cumsum_sig_clch_seen,blue_cumsum_sig_gnd_abs,red_cumsum_sig_gnd_abs,blue_cumsum_sig_gnd_seen,red_cumsum_sig_gnd_seen,blue_cumsum_fight_sec,red_cumsum_fight_sec,blue_cumsum_decWin,red_cumsum_decWin,blue_cumsum_subWin,red_cumsum_subWin,blue_cumsum_KOWin,red_cumsum_KOWin,blue_cumsum_Decloss,red_cumsum_Decloss,blue_cumsum_subLoss,red_cumsum_subLoss,blue_cumsum_KOLoss,red_cumsum_KOLoss,blue_wins,red_wins,blue_losses,red_losses,blue_titleFights,red_titleFights,red_total_stk_per_min,red_sig_stk_per_min,red_total_stk_acc,red_sig_stk_acc,red_stk_head_per_min,red_stk_body_per_min,red_stk_leg_per_min,red_stk_dist_per_min,red_stk_clch_per_min,red_stk_gnd_per_min,red_total_stk_abs_per_min,red_sig_stk_abs_per_min,red_stk_head_abs_per_min,red_stk_body_abs_per_min,red_stk_leg_abs_per_min,red_stk_dist_abs_per_min,red_stk_clch_abs_per_min,red_stk_gnd_abs_per_min,red_total_stk_per_min_diff,red_sig_stk_per_min_diff,red_ctrl_sec_per_min,red_ctrld_sec_per_min,red_ctrl_diff,red_TD_acc,red_TD_def,red_TD_per_min,red_TD_abs_per_min,red_KDs_per_100_stk,red_reach_dif,red_height_dif,blue_total_stk_per_min,blue_sig_stk_per_min,blue_total_stk_acc,blue_sig_stk_acc,blue_stk_head_per_min,blue_stk_body_per_min,blue_stk_leg_per_min,blue_stk_dist_per_min,blue_stk_clch_per_min,blue_stk_gnd_per_min,blue_total_stk_abs_per_min,blue_sig_stk_abs_per_min,blue_stk_head_abs_per_min,blue_stk_body_abs_per_min,blue_stk_leg_abs_per_min,blue_stk_dist_abs_per_min,blue_stk_clch_abs_per_min,blue_stk_gnd_abs_per_min,blue_total_stk_per_min_diff,blue_sig_stk_per_min_diff,blue_ctrl_sec_per_min,blue_ctrld_sec_per_min,blue_ctrl_diff,blue_TD_acc,blue_TD_def,blue_KDs_per_100_stk,blue_reach_dif,blue_height_dif,winner,weight_class,title_fight,prediction,fav_pct,dog_pct,fav_moneyline,dog_moneyline
0,0,30.0,37.0,69,71,Kelvin Gastelum,Jared Cannonier,71,77,1,1,1100,433,2326,763,842,357,2006,681,19,1,51,3,2,0,2251,364,7,4,7,0,165,40,251,63,576,247,1534,640,136,53,162,64,696,253,1686,631,121,39,187,58,60,48,74,7

In [11]:
# convert moneylines to int
future_df["fav_moneyline"] = future_df["fav_moneyline"].astype(int)
future_df["dog_moneyline"] = future_df["dog_moneyline"].astype(int)
future_df

C:\Users\audri\anaconda3\envs\PythonAdv\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\audri\anaconda3\envs\PythonAdv\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,lev1,blue_age,red_age,blue_heightIN,red_heightIN,blue_name,red_name,blue_reach,red_reach,blue_stance,red_stance,blue_cumsum_tot_stks_lnd,red_cumsum_tot_stks_lnd,blue_cumsum_tot_stks_att,red_cumsum_tot_stks_att,blue_cumsum_sig_stks_lnd,red_cumsum_sig_stks_lnd,blue_cumsum_sig_stks_att,red_cumsum_sig_stks_att,blue_cumsum_TDs_lnd,red_cumsum_TDs_lnd,blue_cumsum_TDs_att,red_cumsum_TDs_att,blue_cumsum_sub_att,red_cumsum_sub_att,blue_cumsum_ctrl_sec,red_cumsum_ctrl_sec,blue_cumsum_KDs,red_cumsum_KDs,blue_cumsum_revs,red_cumsum_revs,blue_cumsum_sig_body_lnd,red_cumsum_sig_body_lnd,blue_cumsum_sig_body_att,red_cumsum_sig_body_att,blue_cumsum_sig_head_lnd,red_cumsum_sig_head_lnd,blue_cumsum_sig_head_att,red_cumsum_sig_head_att,blue_cumsum_sig_leg_lnd,red_cumsum_sig_leg_lnd,blue_cumsum_sig_leg_att,red_cumsum_sig_leg_att,blue_cumsum_sig_dist_lnd,red_cumsum_sig_dist_lnd,blue_cumsum_sig_dist_att,red_cumsum_sig_dist_att,blue_cumsum_sig_clch_lnd,red_cumsum_sig_clch_lnd,blue_cumsum_sig_clch_att,red_cumsum_sig_clch_att,blue_cumsum_sig_gnd_lnd,red_cumsum_sig_gnd_lnd,blue_cumsum_sig_gnd_att,red_cumsum_sig_gnd_att,blue_cumsum_tot_stks_abs,red_cumsum_tot_stks_abs,blue_cumsum_tot_stks_seen,red_cumsum_tot_stks_seen,blue_cumsum_sig_stks_abs,red_cumsum_sig_stks_abs,blue_cumsum_sig_stks_seen,red_cumsum_sig_stks_seen,blue_cumsum_TDs_abs,red_cumsum_TDs_abs,blue_cumsum_TDs_seen,red_cumsum_TDs_seen,blue_cumsum_sub_esc,red_cumsum_sub_esc,blue_cumsum_ctrld_sec,red_cumsum_ctrld_sec,blue_cumsum_KedD,red_cumsum_KedD,blue_cumsum_reversed,red_cumsum_reversed,blue_cumsum_sig_body_abs,red_cumsum_sig_body_abs,blue_cumsum_sig_body_seen,red_cumsum_sig_body_seen,blue_cumsum_sig_head_abs,red_cumsum_sig_head_abs,blue_cumsum_sig_head_seen,red_cumsum_sig_head_seen,blue_cumsum_sig_leg_abs,red_cumsum_sig_leg_abs,blue_cumsum_sig_leg_seen,red_cumsum_sig_leg_seen,blue_cumsum_sig_dist_abs,red_cumsum_sig_dist_abs,blue_cumsum_sig_dist_seen,red_cumsum_sig_dist_seen,blue_cumsum_sig_clch_abs,red_cumsum_sig_clch_abs,blue_cumsum_sig_clch_seen,red_cumsum_sig_clch_seen,blue_cumsum_sig_gnd_abs,red_cumsum_sig_gnd_abs,blue_cumsum_sig_gnd_seen,red_cumsum_sig_gnd_seen,blue_cumsum_fight_sec,red_cumsum_fight_sec,blue_cumsum_decWin,red_cumsum_decWin,blue_cumsum_subWin,red_cumsum_subWin,blue_cumsum_KOWin,red_cumsum_KOWin,blue_cumsum_Decloss,red_cumsum_Decloss,blue_cumsum_subLoss,red_cumsum_subLoss,blue_cumsum_KOLoss,red_cumsum_KOLoss,blue_wins,red_wins,blue_losses,red_losses,blue_titleFights,red_titleFights,red_total_stk_per_min,red_sig_stk_per_min,red_total_stk_acc,red_sig_stk_acc,red_stk_head_per_min,red_stk_body_per_min,red_stk_leg_per_min,red_stk_dist_per_min,red_stk_clch_per_min,red_stk_gnd_per_min,red_total_stk_abs_per_min,red_sig_stk_abs_per_min,red_stk_head_abs_per_min,red_stk_body_abs_per_min,red_stk_leg_abs_per_min,red_stk_dist_abs_per_min,red_stk_clch_abs_per_min,red_stk_gnd_abs_per_min,red_total_stk_per_min_diff,red_sig_stk_per_min_diff,red_ctrl_sec_per_min,red_ctrld_sec_per_min,red_ctrl_diff,red_TD_acc,red_TD_def,red_TD_per_min,red_TD_abs_per_min,red_KDs_per_100_stk,red_reach_dif,red_height_dif,blue_total_stk_per_min,blue_sig_stk_per_min,blue_total_stk_acc,blue_sig_stk_acc,blue_stk_head_per_min,blue_stk_body_per_min,blue_stk_leg_per_min,blue_stk_dist_per_min,blue_stk_clch_per_min,blue_stk_gnd_per_min,blue_total_stk_abs_per_min,blue_sig_stk_abs_per_min,blue_stk_head_abs_per_min,blue_stk_body_abs_per_min,blue_stk_leg_abs_per_min,blue_stk_dist_abs_per_min,blue_stk_clch_abs_per_min,blue_stk_gnd_abs_per_min,blue_total_stk_per_min_diff,blue_sig_stk_per_min_diff,blue_ctrl_sec_per_min,blue_ctrld_sec_per_min,blue_ctrl_diff,blue_TD_acc,blue_TD_def,blue_KDs_per_100_stk,blue_reach_dif,blue_height_dif,winner,weight_class,title_fight,prediction,fav_pct,dog_pct,fav_moneyline,dog_moneyline
0,0,30.0,37.0,69,71,Kelvin Gastelum,Jared Cannonier,71,77,1,1,1100,433,2326,763,842,357,2006,681,19,1,51,3,2,0,2251,364,7,4,7,0,165,40,251,63,576,247,1534,640,136,53,162,64,696,253,1686,631,121,39,187,58,60,48,74,7

In [12]:
# load in past picks as we will add our new picks to this running tally
my_picks = pd.read_csv("picks/rolling_picks.csv")
my_picks = my_picks.drop(columns="Unnamed: 0")
my_picks

,blue_name,red_name,prediction,fav_pct,dog_pct,fav_moneyline,dog_moneyline
0,Ciryl Gane,Derrick Lewis,red,0.52,0.48,-108,108
1,Pedro Munhoz,Jose Aldo,red,0.58,0.42,-138,138
2,Vicente Luque,Michael Chiesa,red,0.58,0.42,-138,138
3,Angela Hill,Tecia Torres,red,0.71,0.29,-244,244
4,Casey Kenney,Song Yadong,red,0.63,0.37,-170,170
5,Rafael Fiziev,Bobby Green,blue,0.62,0.38,-163,163
6,Drako Rodriguez,Vince Morales,red,0.63,0.37,-170,170
7,Ed Herman,Alonzo Menifield,blue,0.50,0.50,-100,100
8,Jessica Penne,Karolina Kowalkiewicz,blue,0.54,0.46,-117,117
9,Ode Osbourne,Manel Kape,red,0.58,0.42,-138,138


In [13]:
# dropping everything else to create a clean and simple dataframe of our future predictions 
future_picks = future_df[["blue_name", "red_name", "prediction", "fav_pct", "dog_pct", \
                          "fav_moneyline", "dog_moneyline"]]
future_picks

,blue_name,red_name,prediction,fav_pct,dog_pct,fav_moneyline,dog_moneyline
0,Kelvin Gastelum,Jared Cannonier,red,0.54,0.46,-117,117
1,Mark Madsen,Clay Guida,blue,0.53,0.47,-112,112
2,Chase Sherman,Parker Porter,red,0.50,0.50,-100,100
3,Trevin Jones,Mana Martinez,red,0.60,0.40,-150,150
4,Austin Hubbard,Vinc Pichel,red,0.54,0.46,-117,117
5,Brandon Royval,Alexandre Pantoja,blue,0.51,0.49,-104,104
6,Luis Saldana,Austin Lingo,red,0.59,0.41,-143,143
7,Domingo Pilarte,Brian Kelleher,blue,0.52,0.48,-108,108
8,Josiane Nunes,Bea Malecki,red,0.68,0.32,-212,212
9,Fabio Cherant,William Knight,red,0.62,0.38,-163,163


In [14]:
# This model is my prefered model so i have a running csv of all the predictions made by this model version
my_picks = pd.concat([future_picks, my_picks])
my_picks

,blue_name,red_name,prediction,fav_pct,dog_pct,fav_moneyline,dog_moneyline
0,Kelvin Gastelum,Jared Cannonier,red,0.54,0.46,-117,117
1,Mark Madsen,Clay Guida,blue,0.53,0.47,-112,112
2,Chase Sherman,Parker Porter,red,0.50,0.50,-100,100
3,Trevin Jones,Mana Martinez,red,0.60,0.40,-150,150
4,Austin Hubbard,Vinc Pichel,red,0.54,0.46,-117,117
5,Brandon Royval,Alexandre Pantoja,blue,0.51,0.49,-104,104
6,Luis Saldana,Austin Lingo,red,0.59,0.41,-143,143
7,Domingo Pilarte,Brian Kelleher,blue,0.52,0.48,-108,108
8,Josiane Nunes,Bea Malecki,red,0.68,0.32,-212,212
9,Fabio Cherant,William Knight,red,0.62,0.38,-163,163


In [17]:
my_picks.to_csv("picks/rolling_picks.csv")

In [16]:
# It is important to note that the moneyline is simply a straight calculation of percentage to moneyline, NOT how
# an actual moneyline would be presented at a sports book. Because bookies need to make money, and the moneylines
# they assign can be arbitrary, there will not be a straight 1-1 relationship between the favorite and dog. This is
# because bookies will account for outsides cases of draws and no contests where neither fighter is delared winner.
# It is also the case that bookies are required to make a profit, and will sometimes list BOTH fighters at negative
# money, meaning neither fighter will return more than a dollar for every dollar you bet on them to win. This is 
# seen in fights that are considered a coin flip where the "underdog" will have a moneyline of -110 (or something
# similar) and the "favorite" will be at -115 (or something similar)

# This predictive model SHOULD NOT be used for the purpose of betting without consideration of all these factors,
# and instead should be used as merely one addition datapoint to factor in if you are intending to bet on fights